# Using HarmonicBalance.jl 

We study the single ODE and two coupled ODE examples in the package package [HarmonicBalance.jl](https://github.com/NonlinearOscillations/HarmonicBalance.jl) (see the docstring of the function <i>DifferentialEquation</i>). 

Our <b>goals</b> are to 

1. in the single ODE example replace $x(t)$ by $(x(t))[1]$; 
2. in the two coupled ODEs example replace $[x(t),y(t)]$ by $(x(t))[2]$ or more explicitly, change  

<i> julia> @variables t, x(t), y(t), ω0, ω, F, k; # observe x(t) and y(t) defined seperately </i>

<i> julia> DifferentialEquation(
      [d(x,t,2) + ω0^2 * x - k*y, d(y,t,2) + ω0^2 * y - k*x] .~ [F * cos(ω*t), 0], [x,y] ); </i>
      
to the alternative form

<i> julia> @variables t, (x(t))[1:2], ω0, ω, F, k; # observe x(t) now defines a vector </i>

<i> julia> DifferentialEquation(
      [d(x[1],t,2) + ω0^2 * x[1] - k*x[2], d(x[2],t,2) + ω0^2 * x[2] - k*x[1]] .~ [F * cos(ω*t), 0], [x[1],x[2]] ); </i>

### First Issue: function get_independent_variables()  

Currently the alternative definition of <i>DifferentialEquation</i> fails because of the function <i>get_independent_variables</i>. This failure can be fixed by replacing in this function the line 
(see [this post on Discourse](https://discourse.julialang.org/t/how-to-obtain-val-argument-of-keyset-of-ordercollection/123547/5))

<i>return Num.(flatten(unique([x.val.arguments[1] for x in keys(diff_eom.equations)]))) </i>

by the line

<i>return Num.(unique([arguments(arguments(Symbolics.value(x[1]))[1]) for x in keys(diff_eom.equations)])[1])</i>

### Second Issue: function \_create_harmonic_variable() throws a Method Error when calling the function get_harmonic_equations 


### Longer Term Goals  
 
The broader aim of this exercise is to bring HarmonicBalance.jl to large scale discretized PDE problems.

### Remarks 

1. Beware that copy-and-paste of Latin symbols not always goes well; 

## Import Packages

In [68]:
using SymbolicUtils

In [72]:
var_name(x::SymbolicUtils.Sym) = String(x.name)

var_name (generic function with 1 method)

In [73]:
@syms x::Real

(x,)

In [76]:
String(x.name)

"x"

In [77]:
var_name(Num(x))

LoadError: MethodError: no method matching var_name(::Num)

[0mClosest candidates are:
[0m  var_name([91m::SymbolicUtils.Sym[39m)
[0m[90m   @[39m [36mMain[39m [90m[4mIn[72]:1[24m[39m


In [1]:
using Symbolics
using HarmonicBalance # modified to export _create_harmonic_variable and harmonic_ansatz  
# using Revise 

Precompiling HarmonicBalance
  ✓ HarmonicBalance
  1 dependency successfully precompiled in 49 seconds. 368 already precompiled.


In [2]:
?_create_harmonic_variable

search: _create_harmonic_variable



```julia
_create_harmonic_variable(
    nat_var::Num,
    ω::Num,
    t::Num,
    type::String;
    new_symbol
)

```

Creates a new harmonic variable and its corresponding transformation rule. This function takes a natural variable (`nat_var`), a harmonic frequency (`ω`), an independent variable (`t`), and a type (`type`), and creates a new harmonic variable with the specified `new_symbol`. It returns a tuple containing the transformation rule and the new harmonic variable.


In [42]:
var_name(x::SymbolicUtils.Sym) = String(x.name)

function _my_create_harmonic_variable(
    nat_var::Num, ω::Num, t::Num, type::String; new_symbol::String
)::Tuple{Num,HarmonicVariable}
    new_var = declare_variable(new_symbol, t) # this holds the internal symbol
    name = type * "_{" * var_name(nat_var) * "," * Base.replace(string(ω), "*" => "") * "}"
    # contribution of this harmonic variable to the natural variable
    rule = _coordinate_transform(new_var, ω, t, type)
    hvar = HarmonicVariable(new_var, name, type, ω, nat_var)
    return rule, hvar
end

_my_create_harmonic_variable (generic function with 1 method)

In [4]:
?harmonic_ansatz 

search: harmonic_ansatz get_harmonic_equations



```
harmonic_ansatz(eom::DifferentialEquation, time::Num; coordinates="Cartesian")
```

Expand each variable of `diff_eom` using the harmonics assigned to it with `time` as the time variable. For each harmonic of each variable, instance(s) of `HarmonicVariable` are automatically created and named.


In [5]:
function my_harmonic_ansatz(diff_eom::DifferentialEquation, time::Num)
    ##!is_harmonic(diff_eom, time) &&
    ##    error("The differential equation is not harmonic in ", time, " !")
    eqs = collect(values(diff_eom.equations))
    rules, vars = Dict(), []

    # keep count to label new variables
    uv_idx = 1
    a_idx = 1

    for nvar in get_variables(diff_eom) # sum over natural variables
        to_substitute = Num(0) # combine all the substitution rules for var
        for ω in diff_eom.harmonics[nvar]
            if !isequal(ω, 0) # nonzero harmonic - create u,v
                rule_u, hvar_u = _my_create_harmonic_variable(
                    nvar, ω, time, "u"; new_symbol="u" * string(uv_idx)
                )
                rule_v, hvar_v = _my_create_harmonic_variable(
                    nvar, ω, time, "v"; new_symbol="v" * string(uv_idx)
                )
                rule = rule_u + rule_v
                uv_idx += 1
                push!(vars, hvar_u, hvar_v)
            else # zero harmonic - create a
                rule, hvar = _my_create_harmonic_variable(
                    nvar, ω, time, "a"; new_symbol="a" * string(a_idx)
                )
                a_idx += 1
                push!(vars, hvar)
            end
            to_substitute += rule
        end

        rules[nvar] = to_substitute # total sub rule for nvar
    end
    eqs = substitute_all(eqs, rules)
    return HarmonicEquation(eqs, Vector{HarmonicVariable}(vars), diff_eom)
end

my_harmonic_ansatz (generic function with 1 method)

## Section 1: Introduction

Later more. 

## Section 2: Some Internals of HarmonicBalance  

The package defines a mutable struct DifferentialEquation. 

In [61]:
?DifferentialEquation

search: DifferentialEquation



```julia
mutable struct DifferentialEquation
```

Holds differential equation(s) of motion and a set of harmonics to expand each variable. This is the primary input for `HarmonicBalance.jl`. After inputting the equations, the harmonics ansatz needs to be specified using `add_harmonic!`.

# Fields

  * `equations::OrderedCollections.OrderedDict{Num, Equation}`: Assigns to each variable an equation of motion.
  * `harmonics::OrderedCollections.OrderedDict{Num, OrderedCollections.OrderedSet{Num}}`: Assigns to each variable a set of harmonics.

## Example

```julia-repl
julia> @variables t, x(t), y(t), ω0, ω, F, k;

# equivalent ways to enter the simple harmonic oscillator
julia> DifferentialEquation(d(x,t,2) + ω0^2 * x - F * cos(ω*t), x);
julia> DifferentialEquation(d(x,t,2) + ω0^2 * x ~ F * cos(ω*t), x);

# two coupled oscillators, one of them driven
julia> DifferentialEquation(
    [d(x,t,2) + ω0^2 * x - k*y, d(y,t,2) + ω0^2 * y - k*x] .~ [F * cos(ω*t), 0], [x,y]
);
```


Take the definition of the function flatten() from HarmonicBalance.jl. The package does 

In [62]:
flatten(a) = collect(Iterators.flatten(a))

flatten (generic function with 1 method)

How does get_independent_variables work? 

In [63]:
?get_independent_variables

search: get_independent_variables



```julia
get_independent_variables(
    diff_eom::DifferentialEquation
) -> Any

```

Return the independent dependent variables of `diff_eom`.

---

```julia
get_independent_variables(
    eom::HarmonicEquation
) -> Vector{Num}

```

Return the independent variables (typically time) of `eom`.


## Section 3: Single ODE Example 

We study the single ODE example as a reference for future examples.  

### Single ODE Example Reformulated as Provided by Harmonic Balance 

In [64]:
@variables t, x(t), ω0, ω, F, k;
diff_eq1 = DifferentialEquation(d(x,t,2) + ω0^2 * x - F * cos(ω*t), x)
show(diff_eq1)
add_harmonic!(diff_eq1, x, ω)
show(diff_eq1)
get_harmonic_equations(diff_eq1) 
get_independent_variables(diff_eq1)

"equations"

OrderedCollections.OrderedDict{Num, Equation} with 1 entry:
  x(t) => Differential(t)(Differential(t)(x(t))) - F*cos(t*ω) + x(t)*(ω0^2) ~ 0

"harmonics"

OrderedCollections.OrderedDict{Num, OrderedCollections.OrderedSet{Num}} with 1 entry:
  x(t) => OrderedSet{Num}()

"equations"

OrderedCollections.OrderedDict{Num, Equation} with 1 entry:
  x(t) => Differential(t)(Differential(t)(x(t))) - F*cos(t*ω) + x(t)*(ω0^2) ~ 0

"harmonics"

OrderedCollections.OrderedDict{Num, OrderedCollections.OrderedSet{Num}} with 1 entry:
  x(t) => OrderedSet{Num}([ω])

1-element Vector{Num}:
 t

In [65]:
x.name

LoadError: type Num has no field name

In [10]:
typeof(keys(diff_eq1.equations))

Base.KeySet{Num, OrderedCollections.OrderedDict{Num, Equation}}

In [11]:
harmonic_ansatz(diff_eq1, t) 

A set of 1 harmonic equations
Variables: u1(t), v1(t)
Parameters: ω, F, ω0

Harmonic ansatz: 
x(t) = u1(t)*cos(ωt) + v1(t)*sin(ωt)

Harmonic equations:

Differential(t)(Differential(t)(u1(t)*cos(t*ω) + v1(t)*sin(t*ω))) - F*cos(t*ω) + (u1(t)*cos(t*ω) + v1(t)*sin(t*ω))*(ω0^2) ~ 0


### Single ODE Example Reformulated - Original function get_independent_variables()

In [12]:
@variables t, (x(t))[1], ω0, ω, F, k;
diff_eq2 = DifferentialEquation(d(x[1],t,2) + ω0^2 * x[1] - F * cos(ω*t), x[1])
show(diff_eq2)
get_independent_variables(diff_eq2) 

"equations"

OrderedCollections.OrderedDict{Num, Equation} with 1 entry:
  (x(t))[1] => Differential(t)(Differential(t)((x(t))[1])) - F*cos(t*ω) + (x(t)…

"harmonics"

OrderedCollections.OrderedDict{Num, OrderedCollections.OrderedSet{Num}} with 1 entry:
  (x(t))[1] => OrderedSet{Num}()

2-element Vector{Num}:
 x(t)
    1

In [13]:
keys(diff_eq2.equations)

KeySet for a OrderedCollections.OrderedDict{Num, Equation} with 1 entry. Keys:
  (x(t))[1]

In [14]:
typeof(keys(diff_eq2.equations))

Base.KeySet{Num, OrderedCollections.OrderedDict{Num, Equation}}

In [15]:
Num.(only([arguments(arguments(Symbolics.value(x[1]))[1]) for x in keys(diff_eq2.equations)]))

1-element Vector{Num}:
 t

In [16]:
add_harmonic!(diff_eq2, x[1], ω)
show(diff_eq2)

"equations"

OrderedCollections.OrderedDict{Num, Equation} with 1 entry:
  (x(t))[1] => Differential(t)(Differential(t)((x(t))[1])) - F*cos(t*ω) + (x(t)…

"harmonics"

OrderedCollections.OrderedDict{Num, OrderedCollections.OrderedSet{Num}} with 1 entry:
  (x(t))[1] => OrderedSet{Num}([ω])

### Original function get_independent_variables()

function get_independent_variables(eom::HarmonicEquation)::Vector{Num}
    dynamic_vars = flatten(getfield.(eom.variables, Symbol("symbols")))
    return flatten(unique([SymbolicUtils.arguments(var.val) for var in dynamic_vars]))
end

In [17]:
methods(get_independent_variables)

# 2 methods for generic function "get_independent_variables" from HarmonicBalance:
 [1] get_independent_variables(eom::HarmonicEquation)
     @ ~/.julia/packages/HarmonicBalance/7D2O8/src/HarmonicEquation.jl:225
 [2] get_independent_variables(diff_eom::DifferentialEquation)
     @ ~/.julia/packages/HarmonicBalance/7D2O8/src/DifferentialEquation.jl:136

In [18]:
function get_independent_variables(diff_eom::DifferentialEquation)
    return Num.(flatten(unique([x.val.arguments for x in keys(diff_eom.equations)])))
end

LoadError: invalid method definition in Main: function HarmonicBalance.get_independent_variables must be explicitly imported to be extended

In [19]:
?get_variables

search: get_variables get_cycle_variables get_independent_variables



```
get_variables(e, varlist = nothing; sort::Bool = false)
```

Return a vector of variables appearing in `e`, optionally restricting to variables in `varlist`.

Note that the returned variables are not wrapped in the `Num` type.

# Examples

```julia
julia> @variables t x y z(t);

julia> Symbolics.get_variables(x + y + sin(z); sort = true)
3-element Vector{SymbolicUtils.BasicSymbolic}:
 x
 y
 z(t)

julia> Symbolics.get_variables(x - y; sort = true)
2-element Vector{SymbolicUtils.BasicSymbolic}:
 x
 y
```

---

```julia
get_variables(diff_eom::DifferentialEquation) -> Vector{Num}

```

Return the dependent variables of `diff_eom`.

---

```julia
get_variables(eom::HarmonicEquation) -> Vector{Num}

```

Get the internal symbols of the independent variables of `eom`.


## Section 4: Two-ODE Example as Provided by HarmonicBalance

Run the example provided as example with the documentation of HarmonicBalance.jl. 

In [49]:
@variables t, x(t), y(t), ω0, ω, F, k;

diff_eq3 = DifferentialEquation(
    [d(x,t,2) + ω0^2 * x - k*y, d(y,t,2) + ω0^2 * y - k*x] .~ [F * cos(ω*t), 0], [x,y]
)

System of 2 differential equations
Variables:       x(t), y(t)
Harmonic ansatz: x(t) => ;   y(t) => ;   

Differential(t)(Differential(t)(x(t))) - k*y(t) + x(t)*(ω0^2) ~ F*cos(t*ω)
Differential(t)(Differential(t)(y(t))) - k*x(t) + y(t)*(ω0^2) ~ 0


In [60]:
var_name(x)

LoadError: MethodError: no method matching var_name(::Num)

[0mClosest candidates are:
[0m  var_name([91m::SymbolicUtils.Sym[39m)
[0m[90m   @[39m [36mMain[39m [90m[4mIn[42]:1[24m[39m


In [51]:
typeof(diff_eq3)

DifferentialEquation

In [52]:
show(diff_eq3)

"equations"

OrderedCollections.OrderedDict{Num, Equation} with 2 entries:
  x(t) => Differential(t)(Differential(t)(x(t))) - k*y(t) + x(t)*(ω0^2) ~ F*cos…
  y(t) => Differential(t)(Differential(t)(y(t))) - k*x(t) + y(t)*(ω0^2) ~ 0

"harmonics"

OrderedCollections.OrderedDict{Num, OrderedCollections.OrderedSet{Num}} with 2 entries:
  x(t) => OrderedSet{Num}()
  y(t) => OrderedSet{Num}()

In [53]:
keys(diff_eq3.equations)

KeySet for a OrderedCollections.OrderedDict{Num, Equation} with 2 entries. Keys:
  x(t)
  y(t)

In [54]:
#Num.(only([arguments(arguments(Symbolics.value(x[1]))[1]) for x in keys(diff_eq3.equations)]))

In [55]:
add_harmonic!(diff_eq3, x, ω); 
diff_eq3

System of 2 differential equations
Variables:       x(t), y(t)
Harmonic ansatz: x(t) => ω;   y(t) => ;   

Differential(t)(Differential(t)(x(t))) - k*y(t) + x(t)*(ω0^2) ~ F*cos(t*ω)
Differential(t)(Differential(t)(y(t))) - k*x(t) + y(t)*(ω0^2) ~ 0


In [56]:
add_harmonic!(diff_eq3, y, ω); 
diff_eq3

System of 2 differential equations
Variables:       x(t), y(t)
Harmonic ansatz: x(t) => ω;   y(t) => ω;   

Differential(t)(Differential(t)(x(t))) - k*y(t) + x(t)*(ω0^2) ~ F*cos(t*ω)
Differential(t)(Differential(t)(y(t))) - k*x(t) + y(t)*(ω0^2) ~ 0


In [57]:
harmonic_ansatz(diff_eq3, t)

A set of 2 harmonic equations
Variables: u1(t), v1(t), u2(t), v2(t)
Parameters: ω, k, ω0, F

Harmonic ansatz: 
x(t) = u1(t)*cos(ωt) + v1(t)*sin(ωt)
y(t) = u2(t)*cos(ωt) + v2(t)*sin(ωt)

Harmonic equations:

Differential(t)(Differential(t)(u1(t)*cos(t*ω) + v1(t)*sin(t*ω))) - k*(v2(t)*sin(t*ω) + cos(t*ω)*u2(t)) + (u1(t)*cos(t*ω) + v1(t)*sin(t*ω))*(ω0^2) ~ F*cos(t*ω)

Differential(t)(Differential(t)(v2(t)*sin(t*ω) + cos(t*ω)*u2(t))) - k*(u1(t)*cos(t*ω) + v1(t)*sin(t*ω)) + (v2(t)*sin(t*ω) + cos(t*ω)*u2(t))*(ω0^2) ~ 0


In [29]:
get_harmonic_equations(diff_eq3)

A set of 4 harmonic equations
Variables: u1(T), v1(T), u2(T), v2(T)
Parameters: ω, k, ω0, F

Harmonic ansatz: 
x(t) = u1(T)*cos(ωt) + v1(T)*sin(ωt)
y(t) = u2(T)*cos(ωt) + v2(T)*sin(ωt)

Harmonic equations:

-k*u2(T) + (2//1)*Differential(T)(v1(T))*ω - u1(T)*(ω^2) + u1(T)*(ω0^2) ~ F

-k*v2(T) - (2//1)*Differential(T)(u1(T))*ω - v1(T)*(ω^2) + v1(T)*(ω0^2) ~ 0

-k*u1(T) + (2//1)*Differential(T)(v2(T))*ω - u2(T)*(ω^2) + u2(T)*(ω0^2) ~ 0

-k*v1(T) - (2//1)*Differential(T)(u2(T))*ω - v2(T)*(ω^2) + v2(T)*(ω0^2) ~ 0


In [30]:
get_independent_variables(diff_eq3)

1-element Vector{Num}:
 t

In [31]:
get_variables(diff_eq3)

2-element Vector{Num}:
 x(t)
 y(t)

## Section 5: Two-ODE Example Reformulated  

Make x(t) to be a two-dimensional vector. Remove y(t). 

In [32]:
@variables t, (x(t))[1:2], ω0, ω, F, k; 

Redefine the equations and store output in diff_eq. 

In [33]:
diff_eq4 = DifferentialEquation(
    [d(x[1],t,2) + ω0^2 * x[1] - k*x[2], d(x[2],t,2) + ω0^2 * x[2] - k*x[1]] .~ [F * cos(ω*t), 0], [x[1],x[2]]
)

System of 2 differential equations
Variables:       (x(t))[1], (x(t))[2]
Harmonic ansatz: (x(t))[1] => ;   (x(t))[2] => ;   

Differential(t)(Differential(t)((x(t))[1])) - k*(x(t))[2] + (x(t))[1]*(ω0^2) ~ F*cos(t*ω)
Differential(t)(Differential(t)((x(t))[2])) - k*(x(t))[1] + (x(t))[2]*(ω0^2) ~ 0


In [34]:
show(diff_eq4)

"equations"

OrderedCollections.OrderedDict{Num, Equation} with 2 entries:
  (x(t))[1] => Differential(t)(Differential(t)((x(t))[1])) - k*(x(t))[2] + (x(t…
  (x(t))[2] => Differential(t)(Differential(t)((x(t))[2])) - k*(x(t))[1] + (x(t…

"harmonics"

OrderedCollections.OrderedDict{Num, OrderedCollections.OrderedSet{Num}} with 2 entries:
  (x(t))[1] => OrderedSet{Num}()
  (x(t))[2] => OrderedSet{Num}()

In [35]:
values(diff_eq4.harmonics)

ValueIterator for a OrderedCollections.OrderedDict{Num, OrderedCollections.OrderedSet{Num}} with 2 entries. Values:
  OrderedCollections.OrderedSet{Num}()
  OrderedCollections.OrderedSet{Num}()

In [36]:
get_independent_variables(diff_eq4)

4-element Vector{Num}:
 x(t)
    1
 x(t)
    2

In [37]:
Num.(unique([arguments(arguments(Symbolics.value(x[1]))[1]) for x in keys(diff_eq4.equations)])[1])

1-element Vector{Num}:
 t

In [38]:
get_variables(diff_eq4)

2-element Vector{Num}:
 (x(t))[1]
 (x(t))[2]

In [39]:
add_harmonic!(diff_eq4, x[1], ω); 
diff_eq4

System of 2 differential equations
Variables:       (x(t))[1], (x(t))[2]
Harmonic ansatz: (x(t))[1] => ω;   (x(t))[2] => ;   

Differential(t)(Differential(t)((x(t))[1])) - k*(x(t))[2] + (x(t))[1]*(ω0^2) ~ F*cos(t*ω)
Differential(t)(Differential(t)((x(t))[2])) - k*(x(t))[1] + (x(t))[2]*(ω0^2) ~ 0


In [40]:
add_harmonic!(diff_eq4, x[2], ω); 
diff_eq4

System of 2 differential equations
Variables:       (x(t))[1], (x(t))[2]
Harmonic ansatz: (x(t))[1] => ω;   (x(t))[2] => ω;   

Differential(t)(Differential(t)((x(t))[1])) - k*(x(t))[2] + (x(t))[1]*(ω0^2) ~ F*cos(t*ω)
Differential(t)(Differential(t)((x(t))[2])) - k*(x(t))[1] + (x(t))[2]*(ω0^2) ~ 0


In [48]:
var_name(y(t))

LoadError: Sym y(t) is not callable. Use @syms y(t)(var1, var2,...) to create it as a callable.

In [46]:
var_name(get_variables(diff_eq4)[1])

LoadError: MethodError: no method matching var_name(::Num)

[0mClosest candidates are:
[0m  var_name([91m::SymbolicUtils.Sym[39m)
[0m[90m   @[39m [36mMain[39m [90m[4mIn[42]:1[24m[39m


In [43]:
my_harmonic_ansatz(diff_eq4, t)

LoadError: MethodError: no method matching var_name(::Num)

[0mClosest candidates are:
[0m  var_name([91m::SymbolicUtils.Sym[39m)
[0m[90m   @[39m [36mMain[39m [90m[4mIn[42]:1[24m[39m


In [ ]:
get_harmonic_equations(diff_eq4)

In [ ]:
?harmonic_ansatz

In [ ]:
?_show_ansatz 

In [ ]:
?harmonic_ansatz